# Financial Sentiment Analysis (Single)

In this program, I run a sentiment analysis of a single company based on financial news articles.

The company that I am targeting is Nvidia [NVDA]

## Fetching News Articles

The first step is to fetch the news articles.  

I am using `NewsAPI` to get articles quickly and easily. Then, I use `pandas` to put the articles into a dataframe, where I can collect and read the data easier.  

**Filtering articles:**  
Filter articles that only exist  
- `NewsAPI` sometimes fetches articles that were removed  

**Extracting the data:**  
Extract only the necessary data from the articles
- Title
- Description
- Content

All others can be discarded.  

Both of these steps are part of the cleaning data step that is next in text preprocessing.

In [328]:
import os
from dotenv import load_dotenv

In [329]:
# get path to the environment file
env_path = '../config/.env'
load_dotenv(env_path)

True

In [330]:
# import newsapi package
from newsapi import NewsApiClient

In [331]:
# init newsapi
newsapi = NewsApiClient(api_key=os.getenv('NEWS_API_KEY'))

In [332]:
# fetch all articles that mention Nvidia
all_articles = newsapi.get_everything(q='Nvidia',
                                      language='en')

In [333]:
import pandas as pd
pd.__version__

'2.2.3'

In [334]:
# place all_articles into a dataframe
all_articles_df = pd.DataFrame(all_articles['articles'])
all_articles_df

,source,author,title,description,url,urlToImage,publishedAt,content
0,"{'id': None, 'name': 'Yahoo Entertainment'}",Lawrence Bonk,DOJ subpoenas NVIDIA as part of antitrust prob...,The DOJ has sent subpoenas to NVIDIA and other...,https://consent.yahoo.com/v2/collectConsent?se...,None,2024-09-04T15:34:35Z,"If you click 'Accept all', we and our partners..."
1,"{'id': None, 'name': 'Gizmodo.com'}",Kyle Barr,The Leaked Nvidia RTX 5090 Has So Many Cores I...,Get ready to watch the lights on your block di...,https://gizmodo.com/the-leaked-nvidia-rtx-5090...,https://gizmodo.com/app/uploads/2024/09/Nvidia...,2024-09-27T13:35:22Z,The GeForce RTX 4090 is already so big that an...
2,"{'id': None, 'name': '[Removed]'}",None,[Removed],[Removed],https://removed.com,None,2024-08-29T23:30:37Z,[Removed]
3,"{'id': 'business-insider', 'name': 'Business I...",Emma Cosgrove,Nvidia might actually lose in this key part of...,"As AI matures, Nvidia, Groq, and Cerebras focu...",https://www.businessinsider.com/nvidia-may-los...,https://i.insider.com/66d0c408392a3bda9f2349e3...,2024-09-01T13:00:02Z,Justin Sullivan/Getty\r\n<ul><li>Inference mad...
4,"{'id': 'business-insider', 'name': 'Business I...",Eugene Kim,This chart shows one potential advantage AWS's...,"AI chip investments by Amazon, Google, and Mic...",https://www.businessinsider.com/aws-ai-chips-w...,https://i.insider.com/6622c44b23b29110d3011ce1...,2024-09-26T09:00:02Z,Noah Berger/Getty Images\r\n<ul><li>Big tech c...
...,...,...,...,...,...,...,...,...
95,"{'id': None, 'name': 'Windows Central'}",windowscentral@futurenet.com (WC Staff),What you need to know about AI PCs and how to ...,"AI PCs are a diverse group, and they can come ...",https://www.windowscentral.com/hardware/nvidia...,https://cdn.mos.cms.futurecdn.net/W3JErBHAjSu7...,2024-09-10T13:47:30Z,There has been a surge in AI applications that...
96,"{'id': None, 'name': 'Computerenhance.com'}",Casey Muratori,"Zen, CUDA, and Tensor Cores, Part I: The Silicon",Given the complexity of today's computing hard...,https://www.computerenhance.com/p/zen-cuda-and...,"https://substackcdn.com/image/fetch/w_1200,h_6...",2024-09-03T19:34:32Z,This is part 1 of a miniseries on todays commo...
97,"{'id': None, 'name': 'Hetzner.com'}",None,Hetzner introduces GPU server for AI training,"Find your reliable dream server, optimized for...",https://www.hetzner.com/dedicated-rootserver/g...,None,2024-09-25T11:35:44Z,Discover the next level of performance with th...
98,"{'id': None, 'name': 'Windows Central'}",samuelwtolbert@gmail.com (Samuel Tolbert),"NVIDIA RTX 5080 and 5090 specs seemingly leak,...",The specs for the unrevealed RTX 5080 and 5090...,https://www.windowscentral.com/gaming/pc-gamin...,https://cdn.mos.cms.futurecdn.net/M7rUR7BmMXdM...,2024-09-26T22:22:38Z,What you need to know\r\n<ul><li>The specs for...


In [335]:
# filter articles function
# only filters valid articles
# valid meaning: article exists and description of article exists
def filter_removed_articles(articles):
    return [article for article in articles if article.get('title') != '[Removed]']

In [336]:
# filter the all_articles
valid_articles = filter_removed_articles(all_articles['articles'])

In [337]:
valid_articles_df = pd.DataFrame(valid_articles)
valid_articles_df

,source,author,title,description,url,urlToImage,publishedAt,content
0,"{'id': None, 'name': 'Yahoo Entertainment'}",Lawrence Bonk,DOJ subpoenas NVIDIA as part of antitrust prob...,The DOJ has sent subpoenas to NVIDIA and other...,https://consent.yahoo.com/v2/collectConsent?se...,None,2024-09-04T15:34:35Z,"If you click 'Accept all', we and our partners..."
1,"{'id': None, 'name': 'Gizmodo.com'}",Kyle Barr,The Leaked Nvidia RTX 5090 Has So Many Cores I...,Get ready to watch the lights on your block di...,https://gizmodo.com/the-leaked-nvidia-rtx-5090...,https://gizmodo.com/app/uploads/2024/09/Nvidia...,2024-09-27T13:35:22Z,The GeForce RTX 4090 is already so big that an...
2,"{'id': 'business-insider', 'name': 'Business I...",Emma Cosgrove,Nvidia might actually lose in this key part of...,"As AI matures, Nvidia, Groq, and Cerebras focu...",https://www.businessinsider.com/nvidia-may-los...,https://i.insider.com/66d0c408392a3bda9f2349e3...,2024-09-01T13:00:02Z,Justin Sullivan/Getty\r\n<ul><li>Inference mad...
3,"{'id': 'business-insider', 'name': 'Business I...",Eugene Kim,This chart shows one potential advantage AWS's...,"AI chip investments by Amazon, Google, and Mic...",https://www.businessinsider.com/aws-ai-chips-w...,https://i.insider.com/6622c44b23b29110d3011ce1...,2024-09-26T09:00:02Z,Noah Berger/Getty Images\r\n<ul><li>Big tech c...
4,"{'id': 'business-insider', 'name': 'Business I...",Emma Cosgrove,Nvidia CEO Jensen Huang says the payback on AI...,Nvidia CEO Jensen Huang promised immediate ret...,https://www.businessinsider.com/nvidia-swift-r...,https://i.insider.com/66cfaf8a43b5e59d16b64ee9...,2024-08-29T00:56:26Z,Jensen Huang said demand will continue to be s...
...,...,...,...,...,...,...,...,...
92,"{'id': None, 'name': 'Windows Central'}",windowscentral@futurenet.com (WC Staff),What you need to know about AI PCs and how to ...,"AI PCs are a diverse group, and they can come ...",https://www.windowscentral.com/hardware/nvidia...,https://cdn.mos.cms.futurecdn.net/W3JErBHAjSu7...,2024-09-10T13:47:30Z,There has been a surge in AI applications that...
93,"{'id': None, 'name': 'Computerenhance.com'}",Casey Muratori,"Zen, CUDA, and Tensor Cores, Part I: The Silicon",Given the complexity of today's computing hard...,https://www.computerenhance.com/p/zen-cuda-and...,"https://substackcdn.com/image/fetch/w_1200,h_6...",2024-09-03T19:34:32Z,This is part 1 of a miniseries on todays commo...
94,"{'id': None, 'name': 'Hetzner.com'}",None,Hetzner introduces GPU server for AI training,"Find your reliable dream server, optimized for...",https://www.hetzner.com/dedicated-rootserver/g...,None,2024-09-25T11:35:44Z,Discover the next level of performance with th...
95,"{'id': None, 'name': 'Windows Central'}",samuelwtolbert@gmail.com (Samuel Tolbert),"NVIDIA RTX 5080 and 5090 specs seemingly leak,...",The specs for the unrevealed RTX 5080 and 5090...,https://www.windowscentral.com/gaming/pc-gamin...,https://cdn.mos.cms.futurecdn.net/M7rUR7BmMXdM...,2024-09-26T22:22:38Z,What you need to know\r\n<ul><li>The specs for...


In [338]:
# extract article essentials function
# extract only the title, description, and content from the articles
def extract_article_essentials(articles):
    return [{'title': article['title'], 'content': article['content']} for article in articles]

In [339]:
extracted_articles = extract_article_essentials(valid_articles)

In [340]:
extracted_articles_df = pd.DataFrame(extracted_articles)
extracted_articles_df

,title,content
0,DOJ subpoenas NVIDIA as part of antitrust prob...,"If you click 'Accept all', we and our partners..."
1,The Leaked Nvidia RTX 5090 Has So Many Cores I...,The GeForce RTX 4090 is already so big that an...
2,Nvidia might actually lose in this key part of...,Justin Sullivan/Getty\r\n<ul><li>Inference mad...
3,This chart shows one potential advantage AWS's...,Noah Berger/Getty Images\r\n<ul><li>Big tech c...
4,Nvidia CEO Jensen Huang says the payback on AI...,Jensen Huang said demand will continue to be s...
...,...,...
92,What you need to know about AI PCs and how to ...,There has been a surge in AI applications that...
93,"Zen, CUDA, and Tensor Cores, Part I: The Silicon",This is part 1 of a miniseries on todays commo...
94,Hetzner introduces GPU server for AI training,Discover the next level of performance with th...
95,"NVIDIA RTX 5080 and 5090 specs seemingly leak,...",What you need to know\r\n<ul><li>The specs for...


## Preprocess Text
***This is a crucial***  
Proprocessing helps clean and normalize the text data making it more suitable for analysis.  

After getting the articles, I can now preprocess the text in the articles.  

### Data Cleaning 
**Identify and remove noise:**  
We want to first remove all noise from the data.  
- Punction
- Extra whitespace

**Text normalization:**  
- Stopwords
    - Remove common/irrelevent words that are unlikely to convey much sentiment.  
- Capital letters
    - All letters should be the same case so all words are treated the same in the tokenization process.  

**Data masking:**  
Data masking is not needed in this context.  

Clean text should result.  


In [341]:
# import re package (regular expressions)
import re

In [342]:
import string

In [343]:
# import nltk packages (Natural Language Toolkit)
import nltk
from nltk.corpus import stopwords

In [344]:
# download nltk data packages
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/justinhoang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [345]:
# get all stopwords from data package
stop_words = set(stopwords.words("english"))
print(stop_words)

{'few', 'wouldn', 'being', 'itself', 'we', "hadn't", 'at', 'those', 'through', 'on', 'don', 'yourselves', "needn't", 'have', 'do', 'doing', 'or', "aren't", 'so', 'here', 'wasn', 'o', 'yours', 'what', 'own', "you'll", "doesn't", "shan't", 'such', 'hasn', 'after', 'needn', 'she', 'but', "you've", 'only', 'himself', 'these', 'above', "mustn't", "won't", 'then', 'some', 'how', 'them', 'below', 'both', 'will', 'as', 'its', 'where', 'if', 'myself', 'no', "wasn't", 'ma', 'shouldn', 'has', 'hers', 'aren', 'under', 'he', 'other', 'was', 'there', 'to', 'most', 'up', 'by', 'is', 'ain', "shouldn't", 'why', 'which', 'him', 'me', 'be', 're', 'ourselves', 'mustn', 'it', 'in', 'haven', 'about', "it's", 's', 'should', 'a', 'any', 'until', 'our', 'am', 'off', 'again', "haven't", 't', 'been', "isn't", 'whom', 'weren', 'of', 'who', 'doesn', "you're", 'same', "should've", 'your', 'down', 'd', 'and', 'with', 'just', 'ours', 'yourself', 'm', 'now', 'having', 'couldn', "hasn't", 'herself', "she's", "wouldn't"

In [346]:
# clean text function
# cleans the data (text)
def clean_text(text):
    # remove extra whitespace
    cleaned_text = re.sub(r'\s+', ' ', text).strip()
    # remove HTML tags
    cleaned_text = re.sub(r'<[^>]+>', '', cleaned_text)
    # remove patterns
    cleaned_text = re.sub(r'/\w+\s+\w+', '', cleaned_text)
    # remove URLs
    cleaned_text = re.sub(r'http\S+|www\S+|https\S+', '', cleaned_text, flags=re.MULTILINE)
    # remove punctuation
    cleaned_text = ''.join([char for char in text if char not in string.punctuation])
    
    # lower case all text
    cleaned_text = text.lower()
    # remove stop words from text
    cleaned_text = ' '.join([word for word in cleaned_text.split() if word not in stop_words])
    
    return cleaned_text

In [347]:
columns_to_process = ['title', 'content']

In [348]:
cleaned_articles_df = extracted_articles_df.copy()

In [349]:
for column in columns_to_process:
    cleaned_articles_df[column] = extracted_articles_df[column].apply(clean_text)

In [350]:
cleaned_articles_df.columns = columns_to_process

In [351]:
cleaned_articles_df

,title,content
0,doj subpoenas nvidia part antitrust probe rega...,"click 'accept all', partners, including 239 pa..."
1,leaked nvidia rtx 5090 many cores actually scares,geforce rtx 4090 already big pc builder worth ...
2,nvidia might actually lose key part ai chip bu...,justin sullivan/getty <ul><li>inference made 4...
3,chart shows one potential advantage aws's ai c...,noah berger/getty images <ul><li>big tech clou...
4,nvidia ceo jensen huang says payback ai spendi...,jensen huang said demand continue stronger sup...
...,...,...
92,need know ai pcs choose one,surge ai applications everything generating im...
93,"zen, cuda, tensor cores, part i: silicon",part 1 miniseries todays common consumer compu...
94,hetzner introduces gpu server ai training,discover next level performance new gex130 ded...
95,"nvidia rtx 5080 5090 specs seemingly leak, rev...",need know <ul><li>the specs nvidia's upcoming ...


### Linguistic Proccesing

I break up each of these processes into their own methods to ensure they are being processed correctly.  

**Tokenization**  
Breaking down text into individual words or tokens. Helps separate individual words from the raw text, which makes it easier to analyze and understand.

**POS Tagging**  
Part-of-Speech Tagging

**Lemmatization**  
A technique to reduce words to their root forms based on their part of speech and the meaning of a word based on its context.  
An algorithmic process of identifying an inflected word's "lemma" based on its intended meaning. The algorithm we are utilizing is apart of the `NLTK` library.

**Named-Entity Recoginition**


#### Tokenization

In [352]:
# import nltk tokenization module
from nltk.tokenize import word_tokenize


In [353]:
# import nltk punkt word tokenization data package
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/justinhoang/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [354]:
def tokenize(text):
    return word_tokenize(text)

In [355]:
tokenized_articles_df = cleaned_articles_df.copy()

In [356]:
for column in columns_to_process:
    for index, row in cleaned_articles_df.iterrows():
        try:
            tokenized_articles_df.at[index, column] = tokenize(row[column])
        except Exception as e:
            print(e)

In [357]:
tokenized_articles_df

,title,content
0,"[doj, subpoenas, nvidia, part, antitrust, prob...","[click, 'accept, all, ', ,, partners, ,, inclu..."
1,"[leaked, nvidia, rtx, 5090, many, cores, actua...","[geforce, rtx, 4090, already, big, pc, builder..."
2,"[nvidia, might, actually, lose, key, part, ai,...","[justin, sullivan/getty, <, ul, >, <, li, >, i..."
3,"[chart, shows, one, potential, advantage, aws,...","[noah, berger/getty, images, <, ul, >, <, li, ..."
4,"[nvidia, ceo, jensen, huang, says, payback, ai...","[jensen, huang, said, demand, continue, strong..."
...,...,...
92,"[need, know, ai, pcs, choose, one]","[surge, ai, applications, everything, generati..."
93,"[zen, ,, cuda, ,, tensor, cores, ,, part, i, :...","[part, 1, miniseries, todays, common, consumer..."
94,"[hetzner, introduces, gpu, server, ai, training]","[discover, next, level, performance, new, gex1..."
95,"[nvidia, rtx, 5080, 5090, specs, seemingly, le...","[need, know, <, ul, >, <, li, >, the, specs, n..."


#### POS Tagging

In [358]:
# import nltk pos tagging module
from nltk import pos_tag

In [359]:
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /Users/justinhoang/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [360]:
def pos_tag_text(tokens):
    return pos_tag(tokens)

In [361]:
pos_tagged_articles_df = tokenized_articles_df.copy()

In [362]:
for column in columns_to_process:
    for index, row in tokenized_articles_df.iterrows():
        try:
            pos_tagged_articles_df.at[index, column] = pos_tag_text(row[column])
        except Exception as e:
            print(e)

In [363]:
pos_tagged_articles_df

,title,content
0,"[(doj, NN), (subpoenas, NNS), (nvidia, VBP), (...","[(click, NN), ('accept, IN), (all, DT), (', ''..."
1,"[(leaked, VBN), (nvidia, RB), (rtx, JJ), (5090...","[(geforce, NN), (rtx, NN), (4090, CD), (alread..."
2,"[(nvidia, NN), (might, MD), (actually, RB), (l...","[(justin, NN), (sullivan/getty, JJ), (<, NNP),..."
3,"[(chart, NN), (shows, VBZ), (one, CD), (potent...","[(noah, JJ), (berger/getty, NN), (images, NNS)..."
4,"[(nvidia, RB), (ceo, NN), (jensen, NN), (huang...","[(jensen, NN), (huang, NN), (said, VBD), (dema..."
...,...,...
92,"[(need, NN), (know, VBP), (ai, VBP), (pcs, NNS...","[(surge, NN), (ai, VBP), (applications, NNS), ..."
93,"[(zen, NN), (,, ,), (cuda, NN), (,, ,), (tenso...","[(part, NN), (1, CD), (miniseries, NNS), (toda..."
94,"[(hetzner, NN), (introduces, NNS), (gpu, VBP),...","[(discover, RB), (next, JJ), (level, NN), (per..."
95,"[(nvidia, RB), (rtx, NN), (5080, CD), (5090, C...","[(need, NN), (know, VBP), (<, NNP), (ul, JJ), ..."


#### Lemmatization